In [1]:
import gym
import json
import datetime as dt
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from gym import wrappers

from env.StockTradingEnv import StockTradingEnv
from networks.DQNac import MyModel


In [4]:
df = pd.read_csv('./data/AAPL.csv')
df = df.sort_values('Date')

# The algorithms require a vectorized environment to run
env = StockTradingEnv(df)

base_action = np.array([0,0.5])

obs = env.reset()
for i in range(5):
    action, _states = (base_action, None)
    obs, rewards, done, info = env.step(action)
    
obs=np.expand_dims(obs,axis=0)
obs=np.expand_dims(obs,axis=0)
print(obs)

model = MyModel((1,6,6),32, (1,6), (1,1))
output=model(obs)

[[[[4.17519990e-02 4.32839996e-02 4.27799988e-02 4.11059998e-02
    3.95980010e-02 3.92619996e-02]
   [4.20320008e-02 4.42739990e-02 4.36059998e-02 4.12859986e-02
    3.97299988e-02 3.96140014e-02]
   [4.14619996e-02 4.22600006e-02 4.13480010e-02 4.03260010e-02
    3.85160004e-02 3.88079986e-02]
   [4.17559998e-02 4.26079986e-02 4.16859986e-02 4.08040008e-02
    3.86679992e-02 3.94000000e-02]
   [1.58025418e-02 3.22616659e-02 2.51540449e-02 1.90278981e-02
    2.43973918e-02 1.66822225e-02]
   [2.88633413e-07 4.66440689e-06 2.09547579e-08 4.16896198e-02
    0.00000000e+00 0.00000000e+00]]]]


InvalidArgumentError: in converted code:

    C:\Users\Thomas\PycharmProjects\reinforcement-learning-master\P3A-Deep-Reinforcement-Learning\Stock-Trading-Environment\networks\DQNac.py:35 call  *
        z = self.conv1(z)
    C:\Users\Thomas\PycharmProjects\reinforcement-learning-master\venv\lib\site-packages\tensorflow_core\python\keras\engine\base_layer.py:748 __call__
        self._maybe_build(inputs)
    C:\Users\Thomas\PycharmProjects\reinforcement-learning-master\venv\lib\site-packages\tensorflow_core\python\keras\engine\base_layer.py:2116 _maybe_build
        self.build(input_shapes)
    C:\Users\Thomas\PycharmProjects\reinforcement-learning-master\venv\lib\site-packages\tensorflow_core\python\keras\layers\convolutional.py:158 build
        dtype=self.dtype)
    C:\Users\Thomas\PycharmProjects\reinforcement-learning-master\venv\lib\site-packages\tensorflow_core\python\keras\engine\base_layer.py:446 add_weight
        caching_device=caching_device)
    C:\Users\Thomas\PycharmProje